In [ ]:
import torch
import torch.utils.data as Data
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:
# 使用GPU
device = torch.device("cuda" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [ ]:
# 加载的图片的大小
IMG_SIZE = 64

# 批量大小
BATCH_SIZE = 64

# 可利用的GPU数量，使用0将运行在CPU模式。
ngpu = 1

# 训练次数
num_epochs = 1000

# 训练图片的通道数，彩色图片是3
nc = 3

# 本征向量z的大小(生成器的输入大小)
nz = 100

# 生成器中特征图大小
ngf = 64

# 判别器中特征图大小
ndf = 64

# 优化器学习率
lr = 0.0002

# Adam优化器的Beta1超参
beta1 = 0.5

# 加载数据集

In [ ]:
import torchvision.datasets as dset
import torchvision.transforms as transforms

dataset = dset.ImageFolder(root='./anime-faces',
                           transform=transforms.Compose([
                               transforms.Resize(IMG_SIZE),
                               transforms.CenterCrop(IMG_SIZE),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))

loader = Data.DataLoader(
    dataset = dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2
)

In [ ]:
# 在netG和netD上调用的自定义权重初始化函数
# GAN作者的定义
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
# 生成器代码

class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # 输入是 Z, 对Z进行卷积
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # 输入特征图大小. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # 输入特征图大小. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # 输入特征图大小. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # 输入特征图大小. (ngf) x 32 x 32
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # 输入特征图大小. (nc) x 64 x 64
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # 输入大小 (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # 输入大小. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # 输入大小. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # 输入大小. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
# 创建生成器
generator = Generator(ngpu).to(device)

# 使用权重初始化函数 weights_init 去随机初始化所有权重
#  mean=0, stdev=0.2.
generator.apply(weights_init)

# 输出该模型
print(generator)

In [ ]:
# 创建判别器
discriminator = Discriminator(ngpu).to(device)

# 使用权重初始化函数 weights_init 去随机初始化所有权重
#  mean=0, stdev=0.2.
discriminator.apply(weights_init)

# 输出该模型
print(discriminator)

In [ ]:
# 初始化 BCE损失函数
adversarial_loss = nn.BCELoss().to(device)

# 为G和D都设置Adam优化器
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(beta1, 0.999))
optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(beta1, 0.999))

In [ ]:
for epoch in range(num_epochs):
    # 数据加载器中的每个批次
    for step, (bacth_x,bacth_y) in enumerate(loader):
        
        real_imgs = bacth_x
        # 生成数据的标签
        valid = torch.ones(bacth_x.size(0),1,dtype=torch.float32).to(device) # 真实数据标签为1.0
        fake = torch.zeros(bacth_x.size(0),1,dtype=torch.float32).to(device) # 假数据标签为0.0


        # ---------
        # 生成器
        # ---------
        optimizer_G.zero_grad()

        # 生成输入的灵感 
        z = torch.randn((bacth_x.size(0), nz,1,1)).to(device) # print(z.shape) (64,100)
    
        # 生成器输出
        gen_imgs = generator(z)

        # 生成器代价函数
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)
        g_loss.backward()
        optimizer_G.step()

        # ---------
        #  判别器
        # ---------

        optimizer_D.zero_grad()

        # 计算代价函数
        real_loss = adversarial_loss(discriminator(real_imgs.to(device)), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()
        

    if epoch % 5 ==0:
        print(g_loss)
        print(d_loss)      